In [10]:
from __future__ import annotations

import os, itertools, logging
from typing import Sequence, Tuple, Dict
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

import shap
import psutil

# Torch (RNN path)
from sklearn.neural_network import MLPRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from semopy import Model
pd.set_option("display.max_columns", 300)
logging.basicConfig(level=logging.INFO, format="%(levelname)s | %(message)s")


In [4]:
MODEL = "MLP"   # <-- "MLP" or "RNN"

DATA_PATH = "/Users/arsh/STAT-XAI/Datasets/Synthetic/Loan Approval Categorical Numerical Synthetic Dataset Continous Outcome.csv"  # <- update if needed
TARGET = "Approval_Probability"

NUMERIC_FEATURES = ['Annual_Income','Credit_Score','Employment_Length','Age','Loan_Term']
CATEGORICAL_FEATURES = ['Loan_Purpose','Employment_Status','Loan_Categories','Region','Marital_Status']

TEST_SIZE = 0.30
RANDOM_STATE = 42
THRESHOLD = 0.5
ALPHA = 0.05

# SHAP
SHAP_BATCH_SIZE = 2000
SHAP_PLOT_SAMPLE = 1200

# RNN
EPOCHS = 60
BATCH_TRAIN = 64
BATCH_TEST = 256
EMB_DIM = 16
RNN_HIDDEN = 32
RNN_LAYERS = 1

# Outputs
OUTPUT_DIR = "outputs_mixed"
PLOTS_DIR  = os.path.join(OUTPUT_DIR, "plots")
TABLES_DIR = os.path.join(OUTPUT_DIR, "tables")
SHAP_DIR   = os.path.join(OUTPUT_DIR, "shap")
for d in [OUTPUT_DIR, PLOTS_DIR, TABLES_DIR, SHAP_DIR]:
    os.makedirs(d, exist_ok=True)

print("MODEL:", MODEL)
print("Saving under:", OUTPUT_DIR)


,Annual_Income,Credit_Score,Employment_Length,Age,Loan_Term,Loan_Purpose,Employment_Status,Loan_Categories,Region,Marital_Status,Approval_Probability
0,74944.814262,505.502450,14.599966,30,28,Personal,Self-employed,Medium,West,Married,0.656925
1,144085.716769,483.101653,3.690240,44,14,Home,Employed,Small,South,Divorced,0.719795
2,117839.273017,396.884652,6.932794,55,27,Home,Unemployed,Large,South,Single,0.281247
3,101839.018104,633.996669,13.265613,50,1,Personal,Employed,Small,North,Divorced,0.833786
4,48722.236853,562.143288,9.641787,23,28,Car,Self-employed,Small,South,Single,0.449401


In [12]:
def one_way_anova_eta(df, feat, target):
    formula = f"{target} ~ C({feat})"
    model = ols(formula, data=df).fit()
    aov = sm.stats.anova_lm(model, typ=2)
    ss_factor = aov.loc[f"C({feat})", "sum_sq"]
    ss_total = aov["sum_sq"].sum()
    p_val = aov.loc[f"C({feat})", "PR(>F)"]
    eta_sq = ss_factor / ss_total if ss_total > 0 else np.nan
    return p_val, eta_sq


def statxai_mixed(merged_df: pd.DataFrame,
                  numeric_features: Sequence[str],
                  categorical_features: Sequence[str],
                  model_suffix: str) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    # Ensure cat dtype
    for col in categorical_features:
        if col in merged_df:
            merged_df[col] = merged_df[col].astype('category')

    # 1) MAIN
    main_effects = {}
    
    # Categorical features: one-way ANOVA + η²
    records = []
    
    # Categorical: ANOVA + η²
    for feat in categorical_features:
        p_val, eta_sq = one_way_anova_eta(merged_df, feat, target)
        eff = eta_sq if p_val < ALPHA else 0.0
        main_effects[feat] = eff
        # if p_val < 0.05:
        records.append({
                'Feature': feat,
                'Type': 'Categorical',
                 'p_val': p_val,
                'Effect_Size': eff
            })
    # Numeric features: OLS → standardized beta
    for feat in numeric_features:
        X = sm.add_constant(merged_df[feat])
        y = merged_df[target]
        model = sm.OLS(y, X).fit()
        beta = model.params[feat]
        p_val = model.pvalues[feat]
        # standardized beta = beta * (SD_x / SD_y)
        std_beta = beta * (merged_df[feat].std() / merged_df[target].std())
        eff = abs(std_beta) if p_val < ALPHA else 0.0
        main_effects[feat] = eff
        # if p_val < 0.05:
        records.append({
                'Feature': feat,
                'Type': 'Numeric',
                'p_value': p_val,
                'Effect_Size': abs(eff)
            })

    main_df = (pd.DataFrame(records)
               .sort_values('effect_size', ascending=False)
               .reset_index(drop=True))
    main_df.to_csv(os.path.join(TABLES_DIR, f"main_effects_{model_suffix}.csv"), index=False)

    # 2) PAIRWISE via LRT ΔMcFadden’s R²
    interaction_records = []
    pair_effects = {feat: 0.0 for feat in (numeric_features + categorical_features)}
    
    for f1, f2 in itertools.combinations(numeric_features + categorical_features, 2):
        is_num1 = f1 in numeric_features
        is_num2 = f2 in numeric_features
    
        # Prepare terms for formula
        term1 = f1 if is_num1 else f"C({f1})"
        term2 = f2 if is_num2 else f"C({f2})"
    
        if is_num1 and is_num2:
            # Numeric × Numeric: t-test on interaction coefficient
            full = smf.ols(f"{target} ~ {term1} * {term2}", merged_df).fit()
            p_val = full.pvalues.get(f"{f1}:{f2}", np.nan)
            effect = full.rsquared - smf.ols(f"{target} ~ {term1} + {term2}", merged_df).fit().rsquared
            test = "t-test"
            effect_label = "ΔR²"
        elif not is_num1 and not is_num2:
            # Categorical × Categorical: two-way ANOVA F-test, partial η²
            model = smf.ols(f"{target} ~ {term1} * {term2}", merged_df).fit()
            aov = sm.stats.anova_lm(model, typ=2)
            p_val = aov.loc[f"{term1}:{term2}", "PR(>F)"]
            ss_int = aov.loc[f"{term1}:{term2}", "sum_sq"]
            ss_err = aov.loc["Residual", "sum_sq"]
            effect = ss_int / (ss_int + ss_err)
            test = "F-test"
            effect_label = "ηp²"
        else:
            # Numeric × Categorical: ANCOVA F-test on interaction block, ΔR²
            num_f = f1 if is_num1 else f2
            cat_f = f2 if is_num1 else f1
            full = smf.ols(f"{target} ~ {num_f} * C({cat_f})", merged_df).fit()
            red  = smf.ols(f"{target} ~ {num_f} + C({cat_f})", merged_df).fit()
            aov = sm.stats.anova_lm(red, full)
            p_val = aov["Pr(>F)"].iloc[1]
            effect = full.rsquared - red.rsquared
            test = "F-test"
            effect_label = "ΔR²"
    
        interaction_records.append({
            'Feature Pair':             f"{f1} × {f2}",
            # 'Test':                     test,
            'p-value':                  round(p_val, 4),
            f'Effect Size ({effect_label})': round(effect, 4),
            'Significant?':             'Yes' if p_val < ALPHA else 'No'
        })
        if p_val < ALPHA:
            pair_effects[f1] += effect
            pair_effects[f2] += effect

    pairwise_df = (pd.DataFrame(interaction_records)
                   .sort_values('effect_size_ΔMcF2', ascending=False)
                   .reset_index(drop=True))
    pairwise_df.to_csv(os.path.join(TABLES_DIR, f"pairwise_interactions_{model_suffix}.csv"), index=False)

    # 3) FINAL
    final_records = []
    for feat in features:
        main_eff = main_effects.get(feat, 0.0)
        inter_sum = pair_sums.get(feat, 0.0)
        final_records.append({'Feature': feat,
                              'Main Effect': round(float(main_eff), 6),
                              'Sum Interaction (ΔMcF2)': round(float(inter_sum), 6),
                              'Final Score': round(float(main_eff + inter_sum), 6)})
    final_df = (pd.DataFrame(final_records)
                .sort_values('Final Score', ascending=False)
                .reset_index(drop=True))
    final_df.to_csv(os.path.join(TABLES_DIR, f"final_scores_{model_suffix}.csv"), index=False)

    # Plots
    if not main_df.empty:
        plt.figure()
        plt.bar(main_df['Feature'], main_df['effect_size'])
        plt.xticks(rotation=60, ha='right'); plt.ylabel('Effect Size')
        plt.title(f'{model_suffix}: Main Effects (Descending)')
        plt.tight_layout()
        plt.savefig(os.path.join(PLOTS_DIR, f"main_effects_bar_{model_suffix}.png"), dpi=300)
        plt.show()

    if not pairwise_df.empty:
        dfi = pairwise_df.dropna(subset=['effect_size_ΔMcF2']).head(15)
        if not dfi.empty:
            plt.figure()
            plt.bar(dfi['Feature Pair'], dfi['effect_size_ΔMcF2'])
            plt.xticks(rotation=60, ha='right'); plt.ylabel('ΔMcFadden R²')
            plt.title(f'{model_suffix}: Top Pairwise Effects')
            plt.tight_layout()
            plt.savefig(os.path.join(PLOTS_DIR, f"pairwise_effects_bar_{model_suffix}.png"), dpi=300)
            plt.show()

    if not final_df.empty:
        plt.figure()
        plt.bar(final_df['Feature'], final_df['Final Score'])
        plt.xticks(rotation=60, ha='right'); plt.ylabel('Final Score (main + interactions)')
        plt.title(f'{model_suffix}: Final Scores (Descending)')
        plt.tight_layout()
        plt.savefig(os.path.join(PLOTS_DIR, f"final_effects_bar_{model_suffix}.png"), dpi=300)
        plt.show()

    return main_df, pairwise_df, final_df
